In [28]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [29]:
train_logs = pd.read_csv('data/train_logs.csv')
train_scores = pd.read_csv('data/train_scores.csv')

In [30]:
train = train_logs.merge(train_scores, how="left", on="id")
train.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count,score
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0,3.5
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0,3.5
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0,3.5
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1,3.5
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1,3.5


In [31]:
le = LabelEncoder()
for column in train.columns:
    if train[column].dtype != int:
        train[column] = le.fit_transform(train[column])

In [32]:
train.to_csv('data/train.csv')